# Full Options Screener

In [5]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
from dateutil.relativedelta import relativedelta
import datetime

In [6]:
#Option class
class Option:
    
    def __init__ (self,S,X,T,r,sigma,q,opt_type,price,position,quantity,ticker):
        self.S=S
        self.X=X
        self.T=T
        self.r=r
        self.sigma=sigma
        self.q=q
        self.opt_type=opt_type
        self.price=price
        self.position=position
        self.quantity=quantity
        self.ticker=ticker
    
    #d1 & d2
    def d1(self):
        return (np.log(self.S/self.X)+(self.r-self.q+0.5*np.sqrt(self.sigma))*self.T)/(self.sigma*np.sqrt(self.T))
    def d2(self):
        return self.d1()-self.sigma*np.sqrt(self.T)
    
    #Black Scholes formula
    def bs_price(self):
        if self.opt_type=='Call':
            return self.S*np.exp(-self.q*self.T)*norm.cdf(self.d1())-self.X*np.exp(-self.r*self.T)*norm.cdf(self.d2())
        elif self.opt_type=='Put':
            return -self.S*np.exp(-self.q*self.T)*norm.cdf(-self.d1())+self.X*np.exp(-self.r*self.T)*norm.cdf(-self.d2())
    
    #Greeks
    def delta(self):
        if self.opt_type=='Call':
            return np.exp(-self.q*self.T)*norm.cdf(self.d1())
        elif self.opt_type=='Put':
            return np.exp(-self.q*self.T)*(norm.cdf(self.d1())-1)
    def gamma(self):
        return (norm.pdf(self.d1())*np.exp(-self.q*self.T))/(self.S*self.sigma*np.sqrt(self.T))
    def theta(self):
        if self.opt_type=='Call':
            return (-self.S*norm.pdf(self.d1())*self.sigma*np.exp(-self.q*self.T))/(2*np.sqrt(self.T))+self.q*self.S*norm.cdf(self.d1())*np.exp(-self.q*self.T)-self.r*self.X*np.exp(-self.r*self.T)*norm.cdf(self.d2())
        elif self.opt_type=='Put':
            return (-self.S*norm.pdf(self.d1())*self.sigma*np.exp(-self.q*self.T))/(2*np.sqrt(self.T))-self.q*self.S*norm.cdf(-self.d1())*np.exp(-self.q*self.T)+self.r*self.X*np.exp(-self.r*self.T)*norm.cdf(-self.d2())
    def rho(self):
        if self.opt_type=='Call':
            return self.X*self.T*np.exp(-self.r*self.T)*norm.cdf(self.d2())
        elif self.opt_type=='Put':
            return -self.X*self.T*np.exp(-self.r*self.T)*norm.cdf(-self.d2())
    def vega(self):
        return self.S*np.sqrt(self.T)*norm.pdf(self.d1())*np.exp(-self.q*self.T)
    
    #Implied volatility calculator
    def implied_vol(self):
        vol=np.arange(0.001,30,0.001).tolist()
        array=np.zeros((len(vol),2))
        for t in range(len(vol)):
            array[t,0]=abs(self.price-option_value(self.S,self.X,self.T,self.r,vol[t],self.q,self.opt_type))
            array[t,1]=vol[t]
        df=pd.DataFrame(array,columns=['Epsilon','Vol'])
        return df[df['Epsilon']==df['Epsilon'].min()]['Vol'].iloc[0]
    
    #Sensitivity grapher
    def price_sensitivity(self):
        S_list=np.arange(self.S*0.6,self.S*1.4,self.S*0.02).tolist()
        X_list=np.arange(self.X*0.6,self.X*1.4,self.X*0.02).tolist()
        t_list=np.arange(-self.T,0.0001,self.T*0.02).tolist()
        r_list=np.arange(self.r*0.6,self.r*1.4,self.r*0.02).tolist()
        sigma_list=np.arange(((self.sigma+self.implied_vol())/2)*0.6,((self.sigma+self.implied_vol())/2)*1.4,((self.sigma+self.implied_vol())/2)*0.02).tolist()        
        if self.q!=0:
            q_list=np.arange(self.q*0.6,self.q*1.4,self.q*0.02).tolist()
        
        S_change=[]
        X_change=[]
        t_change=[]
        r_change=[]
        sigma_change=[]
        q_change=[]
        for t in range(len(S_list)):
            S_change.append(option_value(S_list[t], self.X, self.T, self.r, self.sigma, self.q, self.opt_type))
            X_change.append(option_value(self.S, X_list[t], self.T, self.r, self.sigma, self.q, self.opt_type))
            r_change.append(option_value(self.S, self.X, self.T, r_list[t], self.sigma, self.q, self.opt_type))
            sigma_change.append(option_value(self.S, self.X, self.T, self.r, sigma_list[t], self.q, self.opt_type))
            if self.q!=0:
                q_change.append(option_value(self.S, self.X, self.T, self.r, self.sigma, q_list[t], self.opt_type))
        
        for i in range(len(t_list)):
            t_change.append(option_value(self.S, self.X, -t_list[i], self.r, self.sigma, self.q, self.opt_type))
        
       
        print('Sensitivity Analysis for a:', self.opt_type)
        print('S0=', round(self.S,2), 'X=', self.X, 'T=',str(round(self.T,4))+' years','r=',str(self.r*100)+'%','Hist.Volatility=',str(round(self.sigma*100,2))+'%', 'Impl.Volatility=',str(round(self.implied_vol()*100,2))+'%','q=',str(self.q*100)+'%')
        print('Actual price=',str(self.price),'Black & Scholes price=', round(self.bs_price(),3))
        print('Delta: ',str(round(self.delta(),4)), 'Gamma: ',str(round(self.gamma(),5)), 'Theta: ', str(round(self.theta(),4)), 'Rho: ',str(round(self.rho(),4)), 'Vega: ', str(round(self.vega(),4)) )
        fig,axes=plt.subplots(2,3, figsize=(15,6))
        plt.tight_layout()
        
        axes[0,0].plot(S_list, S_change)
        axes[0,0].axvline(x=self.X,linestyle=":",color='black', label='Strike price')
        axes[0,0].axvline(x=self.S,linestyle=":",color='blue',label='Actual price')
        axes[0,0].set_xlabel('S price')
        axes[0,0].set_title('Underlying price sensitivity')
        axes[0,0].legend(loc=0)
        
        axes[0,1].plot(X_list, X_change, color='red')
        axes[0,1].set_xlabel('X price')
        axes[0,1].set_title('Strike price sensitivity')

        axes[0,2].plot(t_list, t_change, color='green')
        axes[0,2].set_xlabel('T-dt')
        axes[0,2].set_title('Time sensitivity')
        
        plt.tight_layout(pad=3)
        axes[1,0].plot(r_list, r_change, color='orange')
        axes[1,0].axvline(x=self.r,linestyle=":",color='black')
        axes[1,0].set_xlabel('r interest rate')
        axes[1,0].set_title('Interest rate sensitivity')
        
        axes[1,1].plot(sigma_list,sigma_change, color='violet')
        axes[1,1].axvline(x=self.sigma,linestyle=":",color='black', label='Hist.vol')
        axes[1,1].axvline(x=self.implied_vol(),linestyle=":",color='blue', label='Impl.vol')
        axes[1,1].set_xlabel('Sigma')
        axes[1,1].set_title('Volatility sensitivity')
        axes[1,1].legend(loc=0)

        if self.q!=0:
            axes[1,2].axvline(x=self.q,linestyle=":",color='grey')
            axes[1,2].plot(q_list, q_change, color='black')
        else:
            axes[1,2].plot()
        axes[1,2].set_xlabel('q rate')
        axes[1,2].set_title('q Sensitivity')
        plt.show()
    
    #Sensitivity grapher for greeks
    def greek_sensitivity(self, greek):
        if greek=='delta':
            n=0
            value='Delta= '+str(round(self.delta(),4))
        if greek=='gamma':
            n=1
            value='Gamma= '+str(round(self.gamma(),4))
        if greek=='theta':
            n=2
            value='Theta= '+str(round(self.theta(),4))
        if greek=='rho':
            n=3
            value='Rho= '+str(round(self.rho(),4))
        if greek=='vega':
            n=4
            value='Vega= '+str(round(self.vega(),4))
            
        S_list=np.arange(self.S*0.6,self.S*1.4,self.S*0.02).tolist()
        X_list=np.arange(self.X*0.6,self.X*1.4,self.X*0.02).tolist()
        t_list=np.arange(-self.T,0.0001,self.T*0.02).tolist()
        r_list=np.arange(self.r*0.6,self.r*1.4,self.r*0.02).tolist()
        sigma_list=np.arange(((self.sigma+self.implied_vol())/2)*0.6,((self.sigma+self.implied_vol())/2)*1.4,((self.sigma+self.implied_vol())/2)*0.02).tolist()
        if self.q!=0:
            q_list=np.arange(self.q*0.6,self.q*1.4,self.q*0.02).tolist()
        
        S_change=[]
        X_change=[]
        t_change=[]
        r_change=[]
        sigma_change=[]
        q_change=[]
        
        for t in range(len(S_list)):
            S_change.append(greeks(S_list[t], self.X, self.T, self.r, self.sigma, self.q, self.opt_type)[n])
            X_change.append(greeks(self.S, X_list[t], self.T, self.r, self.sigma, self.q, self.opt_type)[n])
            r_change.append(greeks(self.S, self.X, self.T, r_list[t], self.sigma, self.q, self.opt_type)[n])
            sigma_change.append(greeks(self.S, self.X, self.T, self.r, sigma_list[t], self.q, self.opt_type)[n])
            if self.q!=0:
                q_change.append(greeks(self.S, self.X, self.T, self.r, self.sigma, q_list[t], self.opt_type)[n])
        
        for i in range(len(t_list)):
            t_change.append(greeks(self.S, self.X, -t_list[i], self.r, self.sigma, self.q, self.opt_type)[n])
        
       
        print( greek ,'sensitivity analysis')
        print(value)
        fig,axes=plt.subplots(2,3, figsize=(15,6))
        plt.tight_layout()
        
        axes[0,0].plot(S_list, S_change)
        axes[0,0].axvline(x=self.X,linestyle=":",color='black', label='Strike price')
        axes[0,0].axvline(x=self.S,linestyle=":",color='blue',label='Actual price')
        axes[0,0].set_xlabel('S price')
        axes[0,0].set_title('Underlying price sensitivity')
        axes[0,0].legend(loc=0)
        
        axes[0,1].plot(X_list, X_change, color='red')
        axes[0,1].set_xlabel('X price')
        axes[0,1].set_title('Strike price sensitivity')

        axes[0,2].plot(t_list, t_change, color='green')
        axes[0,2].set_xlabel('T-dt')
        axes[0,2].set_title('Time sensitivity')
        
        plt.tight_layout(pad=3)
        axes[1,0].plot(r_list, r_change, color='orange')
        axes[1,0].axvline(x=self.r,linestyle=":",color='black')
        axes[1,0].set_xlabel('r interest rate')
        axes[1,0].set_title('Interest rate sensitivity')
        
        axes[1,1].plot(sigma_list,sigma_change, color='violet')
        axes[1,1].axvline(x=self.sigma,linestyle=":",color='black',label='Hist.vol')
        axes[1,1].axvline(x=self.implied_vol(),linestyle=":",color='blue',label='Impl.vol')
        axes[1,1].set_xlabel('Sigma')
        axes[1,1].set_title('Volatility sensitivity')
        axes[1,1].legend(loc=0)

        if self.q!=0:
            axes[1,2].axvline(x=self.q,linestyle=":",color='grey')
            axes[1,2].plot(q_list, q_change, color='black')
        else:
            axes[1,2].plot()
        axes[1,2].set_xlabel('q rate')
        axes[1,2].set_title('q Sensitivity')
        plt.show()
        
    def arbitrage_parity(self,implied_vol):
        if implied_vol==False:
            vol=self.sigma
            msg='Calculations based on historical volatility'
        elif implied_vol==True:
            vol=self.implied_vol()
            msg='Calculations based on implied volatility'
        
        #Arbitrage for r movements
        r_list=np.arange(self.r*0.6,self.r*1.4,self.r*0.02)
        r_difference=[]
        r_parity=[]
        r_bsprice=[]
        for t in range(len(r_list)):
            if self.opt_type=='Call':
                parity_price=option_value(self.S,self.X,self.T,r_list[t],vol,self.q,'Put')+self.S-(self.X/((1+r_list[t])**self.T))
                r_parity.append(parity_price)
            elif self.opt_type=='Put':
                parity_price=option_value(self.S,self.X,self.T,r_list[t],vol,self.q,'Call')-self.S+(self.X/((1+r_list[t])**self.T))
                r_parity.append(parity_price)
            dif=round((option_value(self.S,self.X,self.T,r_list[t],vol,self.q,self.opt_type)-r_parity[t]),2)
            r_difference.append(dif)
            r_bsprice.append(option_value(self.S,self.X,self.T,r_list[t],vol,self.q,self.opt_type))
        
        #Arbitrage for S movements
        S_list=np.arange(self.S*0.6,self.S*1.4,self.S*0.02)
        S_difference=[]
        S_parity=[]
        S_bsprice=[]
        for t in range(len(r_list)):
            if self.opt_type=='Call':
                parity_price=option_value(S_list[t],self.X,self.T,self.r,vol,self.q,'Put')+S_list[t]-(self.X/((1+self.r)**self.T))
                S_parity.append(parity_price)
            elif self.opt_type=='Put':
                parity_price=option_value(S_list[t],self.X,self.T,self.r,vol,self.q,'Call')-S_list[t]+(self.X/((1+self.r)**self.T))
                S_parity.append(parity_price)
            dif=round((option_value(S_list[t],self.X,self.T,self.r,vol,self.q,self.opt_type)-S_parity[t]),2)
            S_difference.append(dif)
            S_bsprice.append(option_value(S_list[t],self.X,self.T,self.r,vol,self.q,self.opt_type))
        
        print(msg)
        fig, axes=plt.subplots(2,2, figsize=(14,9))
        axes[0,0].plot(S_list,S_parity,label='Put-Call Parity price')
        axes[0,0].plot(S_list,S_bsprice,label='BS price')
        axes[0,0].axhline(y=S_bsprice[20],linestyle=":",color='black')
        axes[0,0].axvline(x=self.S,linestyle=":",color='black')
        axes[0,0].set_xlabel('S price')
        axes[0,0].set_title('Arbitrage sensitivity to Underlying price')
        axes[0,0].legend(loc=0)
        axes[0,1].plot(S_list,S_difference,label='Difference')
        axes[0,1].axhline(y=S_difference[20],linestyle=":",color='black')
        axes[0,1].axvline(x=self.S,linestyle=":",color='black')
        axes[0,1].set_xlabel('S price')
        axes[0,1].legend(loc=0)
        
        plt.tight_layout(pad=3)
        axes[1,0].plot(r_list,r_parity,label='Put-Call Parity price')
        axes[1,0].plot(r_list,r_bsprice,label='BS price')
        axes[1,0].axhline(y=r_bsprice[20],linestyle=":",color='black')
        axes[1,0].axvline(x=self.r,linestyle=":",color='black')
        axes[1,0].set_xlabel('r interest rate')
        axes[1,0].set_title('Arbitrage sensitivity to Interest Rate')
        axes[1,0].legend(loc=0)
        axes[1,1].plot(r_list,r_difference,label='Difference')
        axes[1,1].axhline(y=r_difference[20],linestyle=":",color='black')
        axes[1,1].axvline(x=self.r,linestyle=":",color='black')
        axes[1,1].set_xlabel('r interest rate')
        axes[1,1].legend(loc=0)
        plt.show()
        
#Support functions
def option_value(S,X,T,r,sigma, q, opt_type):
    d1=(np.log(S/X)+(r-q+(sigma**2)*0.5)*T)/(sigma*(T**0.5))
    d2=d1-sigma*(T**0.5)
    if opt_type=='Call':
        return S*np.exp(-q*T)*norm.cdf(d1)-X*np.exp(-r*T)*norm.cdf(d2)
    elif opt_type=='Put':
        return -S*np.exp(-q*T)*norm.cdf(-d1)+X*np.exp(-r*T)*norm.cdf(-d2)

def greeks(S,X,T,r,sigma, q, opt_type):
    d1=(np.log(S/X)+(r-q+(sigma**2)/2)*T)/(sigma*(T**0.5))
    d2=d1-sigma*(T**0.5)
    
    if opt_type=='Call':
        delta=np.exp(-q*T)*norm.cdf(d1)
    elif opt_type=='Put':
        delta=np.exp(q*T)*(norm.cdf(d1)-1)
        
    gamma=(norm.pdf(d1)*np.exp(-q*T))/(S*sigma*np.sqrt(T))
    
    if opt_type=='Call':
        theta=(-S*norm.pdf(d1)*sigma*np.exp(-q*T))/(2*np.sqrt(T))+q*S*norm.cdf(d1)*np.exp(-q*T)-r*X*np.exp(-r*T)*norm.cdf(d2)
    elif opt_type=='Put':
        theta= (-S*norm.pdf(d1)*sigma*np.exp(-q*T))/(2*np.sqrt(T))-q*S*norm.cdf(-d1)*np.exp(-q*T)+r*X*np.exp(-r*T)*norm.cdf(-d2)
    
    if opt_type=='Call':
        rho= X*T*np.exp(-r*T)*norm.cdf(d2)
    elif opt_type=='Put':
        rho= -X*T*np.exp(-r*T)*norm.cdf(-d2)
    
    vega=S*np.sqrt(T)*norm.pdf(d1)*np.exp(-q*T)
    
    return delta,gamma,theta,rho,vega


In [7]:
def display_payoff(options):
    opt=np.zeros((40,len(options)+1))
    column=[]
    S_list=np.arange(options[0].S*0.1,options[0].S*1.9,options[0].S*0.02)
    for t in range(len(options)):
        #Long Calls
        if options[t].opt_type=='Call' and options[t].position=='Long':
            column.append('+'+str(options[t].quantity)+'*C'+'('+str(options[t].X)+')')
            for i in range(len(S_list)):
                opt[i,t]=options[t].quantity*(max(S_list[i]-options[t].X,0)-options[t].price)
        #Short Calls
        elif options[t].opt_type=='Call' and options[t].position=='Short':
            column.append('-'+str(options[t].quantity)+'*C'+'('+str(options[t].X)+')')
            for i in range(len(S_list)):
                opt[i,t]=options[t].quantity*(-max(S_list[i]-options[t].X,0)+options[t].price)
        #Long Puts
        elif options[t].opt_type=='Put' and options[t].position=='Long':
            column.append('+'+str(options[t].quantity)+'*P'+'('+str(options[t].X)+')')
            for i in range(len(S_list)):
                opt[i,t]=options[t].quantity*(max(options[t].X-S_list[i],0)-options[t].price)
        #Short Puts
        elif options[t].opt_type=='Put' and options[t].position=='Short':
            column.append('-'+str(options[t].quantity)+'*P'+'('+str(options[t].X)+')')
            for i in range(len(S_list)):
                opt[i,t]=options[t].quantity*(-max(options[t].X-S_list[i],0)+options[t].price)
    
    #Portfolio total gain
    column.append('Value')
    for a in range(0,opt.shape[0]):
        opt[a,-1]=np.sum(opt[a][:-1])
    payoff_df=pd.DataFrame(opt, columns=column, index=S_list)
    payoff_df.plot(figsize=(14,7))
    plt.legend(loc=0)
    plt.xlabel('Stock price')
    plt.ylabel('Gains')
    plt.axhline(y=0,linestyle=":",color='black')
    for k in range (len(options)):
        plt.axvline(x=options[k].X,linestyle=":",color='black')
    plt.show()

In [8]:
class Ulying:
    def __init__(self,ticker):
        self.ticker=ticker
        
    def available_exp(self):
        return yf.Ticker(self.ticker).options

    def available_strike(self,exp_date):
        if self.opt_type=='Call':
            return yf.Ticker(self.ticker).option_chain(exp_date)[0]['strike'].tolist()
        elif self.opt_type=='Put':
            return yf.Ticker(self.ticker).option_chain(exp_date)[1]['strike'].tolist()
        
    def find_option(self,exp_date,strike,opt_type,position='Long',quantity=1):
        start_date=datetime.datetime.strftime(datetime.datetime.now() - relativedelta(years=1),'%Y-%m-%d')
        history=yf.download(self.ticker,start=start_date)['Adj Close']
        
        #S,T & r
        S0=history.iloc[-1]
        sigma=(np.log(history/history.shift(1))).std()*np.sqrt(len(history))
        T=(datetime.datetime.strptime(exp_date, '%Y-%m-%d')-datetime.datetime.now()).days/365
        r=quandl.get("FRED/GS1")['Value'].iloc[-1]/100
        
        #Option price
        if opt_type=='Call':
            price=float(yf.Ticker(self.ticker).option_chain(exp_date)[0]['lastPrice'][yf.Ticker(self.ticker).option_chain(exp_date)[0]['strike']==strike])
            opt_symbol=yf.Ticker(self.ticker).option_chain(exp_date)[0]['contractSymbol'][yf.Ticker(self.ticker).option_chain(exp_date)[0]['strike']==strike].iloc[0]
        elif opt_type=='Put':
            price=float(yf.Ticker(self.ticker).option_chain(exp_date)[1]['lastPrice'][yf.Ticker(self.ticker).option_chain(exp_date)[1]['strike']==strike])
            opt_symbol=yf.Ticker(self.ticker).option_chain(exp_date)[1]['contractSymbol'][yf.Ticker(self.ticker).option_chain(exp_date)[1]['strike']==strike].iloc[0]
        
        return Option(S0,strike,T,r,sigma,0,opt_type,price, position, quantity,opt_symbol)
